In [1]:
import pandas as pd
import numpy as np
#import seaborn as sns
import sqlalchemy
from sqlalchemy.ext.automap import automap_base 
from sqlalchemy.orm import Session
from sqlalchemy import create_engine , func, inspect, MetaData
import json
   
import geojson

In [2]:
data = pd.read_csv('./Data D4H19 - AHT Sept 2018.csv')


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
country_df = pd.read_csv('./country_codes.csv')



In [4]:
data.drop(columns=['By using this data you agree to the Terms of Use: https://www.ctdatacollaborative.org/terms-use'], inplace=True)
data.drop(columns=['yearOfRegistration','Datasource','majorityStatus','majorityStatusAtExploit','majorityEntry'], inplace=True)
data.drop(columns=['isAbduction','typeOfSexConcatenated','typeOfSexPrivateSexualServices','recruiterRelationIntimatePartner','recruiterRelationFriend','RecruiterRelationship','recruiterRelationFamily','recruiterRelationOther','recruiterRelationUnknown'], inplace=True)
data.drop(columns=['typeOfLabourMiningOrDrilling','typeOfLabourPeddling','typeOfLabourOther','typeOfLabourNotSpecified','typeOfLabourConcatenated','typeOfSexProstitution','typeOfSexPornography','typeOfSexRemoteInteractiveServices'], inplace=True)
data.drop(columns=['isForcedLabour','isSexualExploit','isOtherExploit','isSexAndLabour','isOtherExploit','isSexAndLabour','isForcedMarriage','isForcedMilitary','isOrganRemoval'], inplace=True)
data.drop(columns=['typeOfLabourAgriculture','typeOfLabourAquafarming','typeOfLabourBegging','typeOfLabourConstruction','typeOfLabourDomesticWork','typeOfLabourHospitality','typeOfLabourIllicitActivities','typeOfLabourManufacturing','typeOfLabourTransportation'], inplace=True)
data.drop(columns=['typeOfExploitConcatenated','citizenship' ,'meansOfControlConcatenated'], inplace=True)



In [5]:
data_df = data.loc[(
    data["CountryOfExploitation"] != '-99')]

data_df = data_df.loc[(
    data_df["gender"] != '-99')]


In [6]:

meansdata_df = pd.merge(data_df,country_df,left_on='CountryOfExploitation', right_on='country')
meansdata_df.replace(to_replace='-99', value=np.nan, inplace=True)
meansdata_df.replace(to_replace=-99, value=np.nan, inplace=True)

meansdata_country_gender = meansdata_df.copy()




In [7]:
meansdata_df.drop(columns=['country','gender','ageBroad'], inplace=True)


In [8]:
DebtBondage_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlDebtBondage"].rename("DebtBondage")
TakesEarnings_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlTakesEarnings"].rename("TakesEarnings")
RestrictsFinancialAccess_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlRestrictsFinancialAccess"].rename("RestrictsFinancialAccess")
Threats_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlThreats"].rename("Threats")
PsychologicalAbuse_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlPsychologicalAbuse"].rename("PsychologicalAbuse")
PhysicalAbuse_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlPhysicalAbuse"].rename("PhysicalAbuse")
SexualAbuse_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlSexualAbuse"].rename("SexualAbuse")
FalsePromises_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlFalsePromises"].rename("FalsePromises")

PsychoactiveSubstances_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlPsychoactiveSubstances"].rename("PsychoactiveSubstances")
RestrictsMovement_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlRestrictsMovement"].rename("RestrictsMovement")
RestrictsMedicalCare_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlRestrictsMedicalCare"].rename("RestrictsMedicalCare")
ExcessiveWorkingHours_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlExcessiveWorkingHours"].rename("ExcessiveWorkingHours")
UsesChildren_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlUsesChildren"].rename("UsesChildren")
ThreatOfLawEnforcement_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlThreatOfLawEnforcement"].rename("ThreatOfLawEnforcement")
WithholdsNecessities_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlWithholdsNecessities"].rename("WithholdsNecessities")
WithholdsDocuments_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlWithholdsDocuments"].rename("WithholdsDocuments")
Other_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlOther"].rename("Other")
NotSpecified_cnt = meansdata_df.groupby(["CountryOfExploitation"]).count()["meansOfControlNotSpecified"].rename("NotSpecified")



In [9]:
meansofcontrol_df1 = pd.DataFrame({"DebtBondage_cnt": DebtBondage_cnt, "TakesEarnings_cnt":TakesEarnings_cnt, \
                                    "RestrictsFinancialAccess_cnt": RestrictsFinancialAccess_cnt, "Threats_cnt": Threats_cnt, \
                                   "PsychologicalAbuse_cnt": PsychologicalAbuse_cnt, "PhysicalAbuse_cnt":PhysicalAbuse_cnt, \
                                    "SexualAbuse_cnt": SexualAbuse_cnt, "FalsePromises_cnt": FalsePromises_cnt, \
                                   "PsychoactiveSubstances_cnt": PsychoactiveSubstances_cnt, "RestrictsMovement_cnt":RestrictsMovement_cnt, \
                                    "RestrictsMedicalCare_cnt": RestrictsMedicalCare_cnt, "ExcessiveWorkingHours_cnt": ExcessiveWorkingHours_cnt, \
                                   "UsesChildren_cnt": UsesChildren_cnt, "ThreatOfLawEnforcement_cnt":ThreatOfLawEnforcement_cnt, \
                                    "WithholdsNecessities_cnt": WithholdsNecessities_cnt, "WithholdsDocuments_cnt": WithholdsDocuments_cnt, \
                                   "Other_cnt": Other_cnt, "NotSpecified_cnt": NotSpecified_cnt})



In [10]:
col_list= list(meansofcontrol_df1)

col_list.remove('NotSpecified_cnt')
country_totals = []
#meansofcontrol_df['meansofcontrol_total'] = meansofcontrol_df[col_list].sum(axis=1)
country_totals = meansofcontrol_df1[col_list].sum(axis=1)
#total_sum = meansofcontrol_df['meansofcontrol_total'].sum()





In [11]:
meansofcontrol_df1 = meansofcontrol_df1[col_list].apply(lambda x: round((x/x.sum()) * 100 ,2), axis=1) 

meansofcontrol_df1['meansofcontrol_total']  = country_totals
meansofcontrol_df1 = meansofcontrol_df1.fillna(0)

meansofcontrol_df = pd.merge(meansofcontrol_df1,country_df,left_on='CountryOfExploitation', right_on='country')

meansofcontrol_df = meansofcontrol_df.set_index('country')

meansofcontrol_df.head()


,DebtBondage_cnt,TakesEarnings_cnt,RestrictsFinancialAccess_cnt,Threats_cnt,PsychologicalAbuse_cnt,PhysicalAbuse_cnt,SexualAbuse_cnt,FalsePromises_cnt,PsychoactiveSubstances_cnt,RestrictsMovement_cnt,...,ExcessiveWorkingHours_cnt,UsesChildren_cnt,ThreatOfLawEnforcement_cnt,WithholdsNecessities_cnt,WithholdsDocuments_cnt,Other_cnt,meansofcontrol_total,latitude,longitude,name
country,,,,,,,,,,,,,,,,,,,,,
AE,1.07,13.37,0.0,3.21,10.16,4.81,0.53,9.63,0.0,12.3,...,16.04,0.0,1.6,8.02,14.97,0.0,187,23.424076,53.847818,United Arab Emirates
AF,7.50,2.50,0.0,0.00,5.00,2.50,0.00,80.00,0.0,0.0,...,2.50,0.0,0.0,0.00,0.00,0.0,40,33.939110,67.709953,Afghanistan
AL,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,...,0.00,0.0,0.0,0.00,0.00,0.0,0,41.153332,20.168331,Albania
AR,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,...,0.00,0.0,0.0,0.00,0.00,0.0,0,-38.416097,-63.616672,Argentina
AT,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,...,0.00,0.0,0.0,0.00,0.00,0.0,0,47.516231,14.550072,Austria


In [12]:
print(meansofcontrol_df.shape[0])
meansofcontrol_df  = meansofcontrol_df[meansofcontrol_df.meansofcontrol_total != 0]
print(meansofcontrol_df.shape[0])
meansofcontrol_df.head()


54
23


,DebtBondage_cnt,TakesEarnings_cnt,RestrictsFinancialAccess_cnt,Threats_cnt,PsychologicalAbuse_cnt,PhysicalAbuse_cnt,SexualAbuse_cnt,FalsePromises_cnt,PsychoactiveSubstances_cnt,RestrictsMovement_cnt,...,ExcessiveWorkingHours_cnt,UsesChildren_cnt,ThreatOfLawEnforcement_cnt,WithholdsNecessities_cnt,WithholdsDocuments_cnt,Other_cnt,meansofcontrol_total,latitude,longitude,name
country,,,,,,,,,,,,,,,,,,,,,
AE,1.07,13.37,0.0,3.21,10.16,4.81,0.53,9.63,0.00,12.30,...,16.04,0.0,1.6,8.02,14.97,0.00,187,23.424076,53.847818,United Arab Emirates
AF,7.50,2.50,0.0,0.00,5.00,2.50,0.00,80.00,0.00,0.00,...,2.50,0.0,0.0,0.00,0.00,0.00,40,33.939110,67.709953,Afghanistan
BY,0.00,0.00,0.0,16.95,28.81,16.95,30.51,3.39,1.69,1.69,...,0.00,0.0,0.0,0.00,0.00,0.00,59,53.709807,27.953389,Belarus
CN,1.69,0.85,0.0,6.78,1.69,3.39,11.86,27.12,0.00,19.49,...,0.85,0.0,0.0,0.00,5.08,18.64,118,35.861660,104.195397,China
EG,0.00,0.00,0.0,15.94,17.39,15.94,0.00,10.14,0.00,1.45,...,0.00,0.0,0.0,11.59,0.00,14.49,69,26.820553,30.802498,Egypt


In [13]:
#total_sum = meansofcontrol_df['meansofcontrol_total'].sum()

meansofcontrol_sort_df =  meansofcontrol_df.sort_values(by='meansofcontrol_total', ascending=False)

meansofcontrol_sort_df.head()

,DebtBondage_cnt,TakesEarnings_cnt,RestrictsFinancialAccess_cnt,Threats_cnt,PsychologicalAbuse_cnt,PhysicalAbuse_cnt,SexualAbuse_cnt,FalsePromises_cnt,PsychoactiveSubstances_cnt,RestrictsMovement_cnt,...,ExcessiveWorkingHours_cnt,UsesChildren_cnt,ThreatOfLawEnforcement_cnt,WithholdsNecessities_cnt,WithholdsDocuments_cnt,Other_cnt,meansofcontrol_total,latitude,longitude,name
country,,,,,,,,,,,,,,,,,,,,,
US,2.99,6.42,0.79,12.39,13.90,11.49,5.44,3.37,9.48,13.30,...,1.50,0.74,2.18,3.83,2.35,9.14,22305,37.090240,-95.712891,United States
KH,6.63,6.63,0.00,6.73,6.63,6.63,6.99,6.63,6.63,6.63,...,6.63,0.00,6.63,6.63,6.63,6.69,13962,12.565679,104.990963,Cambodia
RU,4.34,11.75,0.00,9.16,10.84,5.55,0.17,11.58,0.18,10.16,...,11.42,0.00,3.92,4.07,10.09,0.08,11032,61.524010,105.318756,Russia
UA,6.05,8.13,0.00,8.59,8.92,8.43,6.55,9.93,3.71,8.89,...,7.70,0.00,4.90,4.09,7.88,0.03,3936,48.379433,31.165580,Ukraine
MY,6.71,6.53,0.00,6.59,6.59,6.65,6.53,7.13,6.53,6.77,...,6.59,0.00,6.53,6.53,6.83,7.01,1655,4.210484,101.975766,Malaysia


In [14]:
    dict = meansofcontrol_sort_df.to_dict(orient='index')
    j = json.dumps({k:{x:y for x,y in v.items() } for k, v in dict.items()})
   
      
    with open('result.json', 'w') as fp:
     json.dump(json.loads(j), fp)
    

In [15]:
meansofcontrol_df.to_csv('meansofcontrol_data.csv')

In [16]:
meansdata_country_gender.drop(columns=['latitude','longitude','name','CountryOfExploitation','country'], inplace=True)

In [17]:
meansdata_country_gender = meansdata_country_gender.loc[(
    meansdata_country_gender["ageBroad"] != 'Unknown')]

meansdata_country_gender = meansdata_country_gender.loc[(
    meansdata_country_gender["ageBroad"].notnull())]

meansdata_country_gender = meansdata_country_gender.loc[(
    meansdata_country_gender["gender"] != 'Unknown')]

len(meansdata_country_gender)

31476

In [18]:
def age_category (row):
   if row['ageBroad'] == '0--8' :
      return 'child'
   elif row['ageBroad'] == '9--17':
      return 'child'
   else: 
      return 'adult'

In [19]:
meansdata_country_gender['age_category'] = meansdata_country_gender.apply (lambda row: age_category(row), axis=1)

In [20]:
meansdata_country_gender.to_csv('meansdata_country_gender.csv')

In [21]:
DebtBondage_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlDebtBondage"].rename("DebtBondage")
TakesEarnings_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlTakesEarnings"].rename("TakesEarnings")
RestrictsFinancialAccess_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlRestrictsFinancialAccess"].rename("RestrictsFinancialAccess")
Threats_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlThreats"].rename("Threats")
PsychologicalAbuse_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlPsychologicalAbuse"].rename("PsychologicalAbuse")
PhysicalAbuse_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlPhysicalAbuse"].rename("PhysicalAbuse")
SexualAbuse_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlSexualAbuse"].rename("SexualAbuse")
FalsePromises_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlFalsePromises"].rename("FalsePromises")

PsychoactiveSubstances_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlPsychoactiveSubstances"].rename("PsychoactiveSubstances")
RestrictsMovement_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlRestrictsMovement"].rename("RestrictsMovement")
RestrictsMedicalCare_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlRestrictsMedicalCare"].rename("RestrictsMedicalCare")
ExcessiveWorkingHours_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlExcessiveWorkingHours"].rename("ExcessiveWorkingHours")
UsesChildren_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlUsesChildren"].rename("UsesChildren")
ThreatOfLawEnforcement_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlThreatOfLawEnforcement"].rename("ThreatOfLawEnforcement")
WithholdsNecessities_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlWithholdsNecessities"].rename("WithholdsNecessities")
WithholdsDocuments_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlWithholdsDocuments"].rename("WithholdsDocuments")
Other_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlOther"].rename("Other")
NotSpecified_ct = meansdata_country_gender.groupby(["age_category"]).count()["meansOfControlNotSpecified"].rename("NotSpecified")




In [22]:
meansdata_age = pd.DataFrame({"Debt Bondage": DebtBondage_ct, "Takes Earnings":TakesEarnings_ct, \
                                    "Restricts Financial Access": RestrictsFinancialAccess_ct, "Threats": Threats_ct, \
                                   "Psychological Abuse": PsychologicalAbuse_ct, "Physical Abuse":PhysicalAbuse_ct, \
                                    "Sexual Abuse": SexualAbuse_ct, "False Promises": FalsePromises_ct, \
                                   "Psychoactive Substances": PsychoactiveSubstances_ct, "Restricts Movement":RestrictsMovement_ct, \
                                    "Restricts Medical Care": RestrictsMedicalCare_ct, "Excessive WorkingHours": ExcessiveWorkingHours_ct, \
                                   "Uses Children": UsesChildren_ct, "Threat Of Law Enforcement":ThreatOfLawEnforcement_ct, \
                                    "Withholds Necessities": WithholdsNecessities_ct, "Withholds Documents": WithholdsDocuments_ct, \
                                   "Other": Other_ct})

meansdata_age = meansdata_age.apply(lambda x: round((x/x.sum()) * 100 ,2), axis=1) 

meansdata_age

,Debt Bondage,Takes Earnings,Restricts Financial Access,Threats,Psychological Abuse,Physical Abuse,Sexual Abuse,False Promises,Psychoactive Substances,Restricts Movement,Restricts Medical Care,Excessive WorkingHours,Uses Children,Threat Of Law Enforcement,Withholds Necessities,Withholds Documents,Other
age_category,,,,,,,,,,,,,,,,,
adult,4.37,8.73,0.27,9.55,10.85,8.38,3.42,8.09,4.37,10.64,4.85,7.24,0.28,3.29,4.50,7.09,4.07
child,4.59,5.12,0.07,8.64,12.14,9.46,7.54,6.25,9.19,8.75,3.99,4.38,0.01,3.97,5.16,3.85,6.88


In [23]:
 meansdata_age_transposed =meansdata_age.T


In [24]:
 meansdata_age_transposed .to_csv('meansdata_age.csv')

In [25]:
meansdata_age_transposed

age_category,adult,child
Debt Bondage,4.37,4.59
Takes Earnings,8.73,5.12
Restricts Financial Access,0.27,0.07
Threats,9.55,8.64
Psychological Abuse,10.85,12.14
Physical Abuse,8.38,9.46
Sexual Abuse,3.42,7.54
False Promises,8.09,6.25
Psychoactive Substances,4.37,9.19
Restricts Movement,10.64,8.75


In [26]:
# Create an empty list 
Row_list =[] 
  
# Iterate over each row 
for rows in meansdata_age_transposed.itertuples(): 
    # Create list for the current row 
    my_list =[ rows.Index,rows.adult, rows.child] 
      
    # append the list to the final list 
    Row_list.append(my_list) 
  
# Print the list 
print(Row_list) 

[['Debt Bondage', 4.37, 4.59], ['Takes Earnings', 8.73, 5.12], ['Restricts Financial Access', 0.27, 0.07], ['Threats', 9.55, 8.64], ['Psychological Abuse', 10.85, 12.14], ['Physical Abuse', 8.38, 9.46], ['Sexual Abuse', 3.42, 7.54], ['False Promises', 8.09, 6.25], ['Psychoactive Substances', 4.37, 9.19], ['Restricts Movement', 10.64, 8.75], ['Restricts Medical Care', 4.85, 3.99], ['Excessive WorkingHours', 7.24, 4.38], ['Uses Children', 0.28, 0.01], ['Threat Of Law Enforcement', 3.29, 3.97], ['Withholds Necessities', 4.5, 5.16], ['Withholds Documents', 7.09, 3.85], ['Other', 4.07, 6.88]]


In [27]:
DebtBondage_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlDebtBondage"].rename("DebtBondage")
TakesEarnings_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlTakesEarnings"].rename("TakesEarnings")
RestrictsFinancialAccess_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlRestrictsFinancialAccess"].rename("RestrictsFinancialAccess")
Threats_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlThreats"].rename("Threats")
PsychologicalAbuse_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlPsychologicalAbuse"].rename("PsychologicalAbuse")
PhysicalAbuse_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlPhysicalAbuse"].rename("PhysicalAbuse")
SexualAbuse_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlSexualAbuse"].rename("SexualAbuse")
FalsePromises_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlFalsePromises"].rename("FalsePromises")

PsychoactiveSubstances_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlPsychoactiveSubstances"].rename("PsychoactiveSubstances")
RestrictsMovement_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlRestrictsMovement"].rename("RestrictsMovement")
RestrictsMedicalCare_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlRestrictsMedicalCare"].rename("RestrictsMedicalCare")
ExcessiveWorkingHours_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlExcessiveWorkingHours"].rename("ExcessiveWorkingHours")
UsesChildren_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlUsesChildren"].rename("UsesChildren")
ThreatOfLawEnforcement_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlThreatOfLawEnforcement"].rename("ThreatOfLawEnforcement")
WithholdsNecessities_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlWithholdsNecessities"].rename("WithholdsNecessities")
WithholdsDocuments_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlWithholdsDocuments"].rename("WithholdsDocuments")
Other_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlOther"].rename("Other")
NotSpecified_ct1 = meansdata_country_gender.groupby(["gender"]).count()["meansOfControlNotSpecified"].rename("NotSpecified")



In [28]:
meansdata_gender = pd.DataFrame({"Debt Bondage": DebtBondage_ct1, "Takes Earnings":TakesEarnings_ct1, \
                                    "Restricts Financial Access": RestrictsFinancialAccess_ct1, "Threats": Threats_ct1, \
                                   "Psychological Abuse": PsychologicalAbuse_ct1, "Physical Abuse":PhysicalAbuse_ct1, \
                                    "Sexual Abuse": SexualAbuse_ct1, "False Promises": FalsePromises_ct1, \
                                   "Psychoactive Substances": PsychoactiveSubstances_ct1, "Restricts Movement":RestrictsMovement_ct1, \
                                    "Restricts MedicalCare": RestrictsMedicalCare_ct1, "Excessive Working Hours": ExcessiveWorkingHours_ct1, \
                                   "Uses Children": UsesChildren_ct1, "Threat Of Law Enforcement":ThreatOfLawEnforcement_ct1, \
                                    "Withholds Necessities": WithholdsNecessities_ct1, "Withholds Documents": WithholdsDocuments_ct1, \
                                   "Other": Other_ct1})

meansdata_gender = meansdata_gender.apply(lambda x: round((x/x.sum()) * 100 ,2), axis=1) 

meansdata_gender


,Debt Bondage,Takes Earnings,Restricts Financial Access,Threats,Psychological Abuse,Physical Abuse,Sexual Abuse,False Promises,Psychoactive Substances,Restricts Movement,Restricts MedicalCare,Excessive Working Hours,Uses Children,Threat Of Law Enforcement,Withholds Necessities,Withholds Documents,Other
gender,,,,,,,,,,,,,,,,,
Female,4.29,6.49,0.36,9.99,11.89,10.29,5.81,5.80,7.46,10.82,3.72,4.47,0.35,2.94,4.17,5.08,6.07
Male,4.64,10.42,0.00,8.30,9.85,5.80,1.78,10.87,1.96,9.26,6.25,10.24,0.00,4.26,5.43,8.60,2.34


In [29]:
 meansdata_gender_transposed = meansdata_gender.T

In [30]:
meansdata_gender_transposed['Female'] = meansdata_gender_transposed['Female'] * -1


In [31]:
meansdata_gender_transposed

gender,Female,Male
Debt Bondage,-4.29,4.64
Takes Earnings,-6.49,10.42
Restricts Financial Access,-0.36,0.00
Threats,-9.99,8.30
Psychological Abuse,-11.89,9.85
Physical Abuse,-10.29,5.80
Sexual Abuse,-5.81,1.78
False Promises,-5.80,10.87
Psychoactive Substances,-7.46,1.96
Restricts Movement,-10.82,9.26


In [32]:
# Create an empty list 
Row_lst =[] 
  
# Iterate over each row 
for rows in meansdata_gender_transposed.itertuples(): 
    # Create list for the current row 
    my_lst =[ rows.Index,rows.Male, rows.Female] 
      
    # append the list to the final list 
    Row_lst.append(my_lst) 
  
# Print the list 
print(Row_lst) 

[['Debt Bondage', 4.64, -4.29], ['Takes Earnings', 10.42, -6.49], ['Restricts Financial Access', 0.0, -0.36], ['Threats', 8.3, -9.99], ['Psychological Abuse', 9.85, -11.89], ['Physical Abuse', 5.8, -10.29], ['Sexual Abuse', 1.78, -5.81], ['False Promises', 10.87, -5.8], ['Psychoactive Substances', 1.96, -7.46], ['Restricts Movement', 9.26, -10.82], ['Restricts MedicalCare', 6.25, -3.72], ['Excessive Working Hours', 10.24, -4.47], ['Uses Children', 0.0, -0.35], ['Threat Of Law Enforcement', 4.26, -2.94], ['Withholds Necessities', 5.43, -4.17], ['Withholds Documents', 8.6, -5.08], ['Other', 2.34, -6.07]]


In [33]:
meansdata_gender_transposed.to_csv('meansdata_gender.csv')

In [34]:
engine = create_engine("sqlite:///meansofcontrol.sqlite" ,echo=False)
conn = engine.connect()


In [35]:
engine.execute('delete FROM  meansofcontrol')

engine.execute('delete FROM  meansdata_age')

engine.execute('delete FROM  meansdata_gender')

In [36]:
meansofcontrol_df.to_sql("meansofcontrol", conn, if_exists="append")

meansdata_age_transposed.to_sql("meansdata_age", conn, if_exists="append")

meansdata_gender_transposed.to_sql("meansdata_gender", conn, if_exists="append")



In [37]:
inspector = inspect(engine)
inspector.get_table_names()
columns = inspector.get_columns('meansofcontrol')

columns

[{'name': 'country',
  'type': TEXT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 1},
 {'name': 'DebtBondage_cnt',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'TakesEarnings_cnt',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'RestrictsFinancialAccess_cnt',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'Threats_cnt',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'PsychologicalAbuse_cnt',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'PhysicalAbuse_cnt',
  'type': FLOAT(),
  'nullable': True,
  'default': None,
  'autoincrement': 'auto',
  'primary_key': 0},
 {'name': 'SexualAbuse_cnt',
  'type': FLOAT

In [38]:
Base = automap_base()
Base.prepare(engine, reflect=True)

Base.classes.keys()
Meansofcontrol = Base.classes.meansofcontrol


In [41]:
engine.execute('SELECT  *  FROM  meansofcontrol order by meansofcontrol_total desc ').fetchall()

[('US', 2.99, 6.42, 0.79, 12.39, 13.9, 11.49, 5.44, 3.37, 9.48, 13.3, 0.69, 1.5, 0.74, 2.18, 3.83, 2.35, 9.14, 22305, 37.09024, -95.712891, 'United States'),
 ('KH', 6.63, 6.63, 0.0, 6.73, 6.63, 6.63, 6.99, 6.63, 6.63, 6.63, 6.63, 6.63, 0.0, 6.63, 6.63, 6.63, 6.69, 13962, 12.565679, 104.990963, 'Cambodia'),
 ('RU', 4.34, 11.75, 0.0, 9.16, 10.84, 5.55, 0.17, 11.58, 0.18, 10.16, 6.69, 11.42, 0.0, 3.92, 4.07, 10.09, 0.08, 11032, 61.52401, 105.31875600000001, 'Russia'),
 ('UA', 6.05, 8.13, 0.0, 8.59, 8.92, 8.43, 6.55, 9.93, 3.71, 8.89, 6.2, 7.7, 0.0, 4.9, 4.09, 7.88, 0.03, 3936, 48.379433, 31.16558, 'Ukraine'),
 ('MY', 6.71, 6.53, 0.0, 6.59, 6.59, 6.65, 6.53, 7.13, 6.53, 6.77, 6.53, 6.59, 0.0, 6.53, 6.53, 6.83, 7.01, 1655, 4.210483999999999, 101.97576600000001, 'Malaysia'),
 ('KZ', 5.72, 11.9, 0.0, 10.53, 7.89, 6.98, 1.49, 11.9, 0.23, 10.64, 7.89, 8.81, 0.0, 2.29, 8.01, 5.72, 0.0, 874, 48.019573, 66.923684, 'Kazakhstan'),
 ('ID', 0.69, 11.22, 0.0, 3.05, 10.25, 4.16, 0.0, 12.19, 0.42, 13.57

In [42]:
engine.execute('SELECT  * FROM  meansdata_age ').fetchall()

[('Debt Bondage', 4.37, 4.59),
 ('Takes Earnings', 8.73, 5.12),
 ('Restricts Financial Access', 0.27, 0.07),
 ('Threats', 9.55, 8.64),
 ('Psychological Abuse', 10.85, 12.14),
 ('Physical Abuse', 8.38, 9.46),
 ('Sexual Abuse', 3.42, 7.54),
 ('False Promises', 8.09, 6.25),
 ('Psychoactive Substances', 4.37, 9.19),
 ('Restricts Movement', 10.64, 8.75),
 ('Restricts Medical Care', 4.85, 3.99),
 ('Excessive WorkingHours', 7.24, 4.38),
 ('Uses Children', 0.28, 0.01),
 ('Threat Of Law Enforcement', 3.29, 3.97),
 ('Withholds Necessities', 4.5, 5.16),
 ('Withholds Documents', 7.09, 3.85),
 ('Other', 4.07, 6.88)]

In [43]:
engine.execute('SELECT  * FROM  meansdata_gender ').fetchall()

[('Debt Bondage', -4.29, 4.64),
 ('Takes Earnings', -6.49, 10.42),
 ('Restricts Financial Access', -0.36, 0.0),
 ('Threats', -9.99, 8.3),
 ('Psychological Abuse', -11.89, 9.85),
 ('Physical Abuse', -10.29, 5.8),
 ('Sexual Abuse', -5.81, 1.78),
 ('False Promises', -5.8, 10.87),
 ('Psychoactive Substances', -7.46, 1.96),
 ('Restricts Movement', -10.82, 9.26),
 ('Restricts MedicalCare', -3.72, 6.25),
 ('Excessive Working Hours', -4.47, 10.24),
 ('Uses Children', -0.35, 0.0),
 ('Threat Of Law Enforcement', -2.94, 4.26),
 ('Withholds Necessities', -4.17, 5.43),
 ('Withholds Documents', -5.08, 8.6),
 ('Other', -6.07, 2.34)]

In [44]:
col_list= list(meansofcontrol_df)

col_list



['DebtBondage_cnt',
 'TakesEarnings_cnt',
 'RestrictsFinancialAccess_cnt',
 'Threats_cnt',
 'PsychologicalAbuse_cnt',
 'PhysicalAbuse_cnt',
 'SexualAbuse_cnt',
 'FalsePromises_cnt',
 'PsychoactiveSubstances_cnt',
 'RestrictsMovement_cnt',
 'RestrictsMedicalCare_cnt',
 'ExcessiveWorkingHours_cnt',
 'UsesChildren_cnt',
 'ThreatOfLawEnforcement_cnt',
 'WithholdsNecessities_cnt',
 'WithholdsDocuments_cnt',
 'Other_cnt',
 'meansofcontrol_total',
 'latitude',
 'longitude',
 'name']

In [45]:
# create the function
def df_to_geojson(df, properties, latitude='latitude', longitude='longitude'):
    """
    Turn a dataframe containing point data into a geojson formatted python dictionary

    df : the dataframe to convert to geojson
    properties : a list of columns in the dataframe to turn into geojson feature properties
    lat : the name of the column in the dataframe that contains latitude data
    lng : the name of the column in the dataframe that contains longitude data
    """

    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    # x is the equivalent of the row, df.iterrows converts the dataframe in to a pd.series object
    # the x is a counter and has no influence
    for x, row in df.iterrows():

        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [float(row.longitude),float(row.latitude)]

        # be aware that the dataframe is a pd.series
        #print('rowitem converts to ndarray(numpy) :\n ', row)
        # convert the array to a pandas.serie
        geo_props = pd.Series(row)

        # for each column, get the value and add it as a new feature property
        # prop determines the list from the properties
        for prop in properties:

            #loop over the items to convert to string elements

            #convert to string
            if type(geo_props[prop]) == float:
                #print('ok')
                geo_props[prop] = str(int(geo_props[prop]))

            # now create a json format, here we have to make the dict properties
            feature['properties'][prop] = geo_props[prop]

        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    return geojson

In [46]:
df_to_geojson(meansofcontrol_sort_df,col_list,'latitude','longitude')

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'DebtBondage_cnt': '2',
    'TakesEarnings_cnt': '6',
    'RestrictsFinancialAccess_cnt': '0',
    'Threats_cnt': '12',
    'PsychologicalAbuse_cnt': '13',
    'PhysicalAbuse_cnt': '11',
    'SexualAbuse_cnt': '5',
    'FalsePromises_cnt': '3',
    'PsychoactiveSubstances_cnt': '9',
    'RestrictsMovement_cnt': '13',
    'RestrictsMedicalCare_cnt': '0',
    'ExcessiveWorkingHours_cnt': '1',
    'UsesChildren_cnt': '0',
    'ThreatOfLawEnforcement_cnt': '2',
    'WithholdsNecessities_cnt': '3',
    'WithholdsDocuments_cnt': '2',
    'Other_cnt': '9',
    'meansofcontrol_total': 22305,
    'latitude': '37',
    'longitude': '-95',
    'name': 'United States'},
   'geometry': {'type': 'Point', 'coordinates': [-95.712891, 37.09024]}},
  {'type': 'Feature',
   'properties': {'DebtBondage_cnt': '6',
    'TakesEarnings_cnt': '6',
    'RestrictsFinancialAccess_cnt': '0',
    'Threats_cnt': '6',
    'Psychological

In [47]:
with open('map1.geojson','w') as f:
    json.dump(df_to_geojson(meansofcontrol_sort_df,col_list,'latitude','longitude'),f)